# GPKit

This notebook is a recallection of the algorithms and functions used to analyse and simulate the GPK algorithm.

## The GPK

The GPK is a quantum algorithm that is applied to a Boolean function $f:\mathbb{F}_2^n\to\mathbb{F}_2^m$, so in order to properly simulate it, we will need some building blocks for working with the vectors of $\mathbb{F}_2^n$. We will call these vectors Boolean vectors or binary strings idistinctly and we will represent them as strings. For example:
$$\mathbf{x} = 01001.$$
These basic operations are the following. Given two Boolean vectors $\mathbf{x}, \mathbf{y}\in \mathbb{F}_2^n$, written as
$$
\mathbf{x} = x_{n-1}\ldots x_{1}x_0\quad\quad \mathbf{y} = y_{n-1}\ldots y_1 y_0,
$$
the sum in $\mathbb{F}_2^n$, which is the bitwise exclusive or, is:
$$
\mathbf{x}+\mathbf{y} = (x_{n-1}\oplus y_{n-1})\ldots (x_1\oplus y_1)(x_0\oplus y_0).
$$
The inner product in $\mathbb{F}_2^n$ is,
$$
\mathbf{x}\cdot\mathbf{y} = (x_{n-1}\cdot y_{n-1})\oplus\ldots(x_1\cdot y_1)\oplus(x_0\cdot y_0).
$$
We will also include a function that converts an integer into the string associated with its base $2$ form and another that visits all the elements of $\mathbb{F}_2^n$ in lexicographical order.

In [16]:
import numpy as np
import math

# Bitwise sum of two strings x and y.

def sums(x,y):
    k = len(x)
    S = ''
    for i in range(0,k):
        z = (int(x[i]) + int(y[i])) % 2
        S = S + str(z)
    return(S)

# Product of strings x and y.

def prods(x,y):
    k = len(x)
    S = 0
    for i in range(0,k):
        z = (int(x[i])*int(y[i]))
        S = (S + z) % 2
    return(S)

# Converts an integer i into a binary n-string.

def bins(i,n):
    S = ''
    for j in range(1,n+1):
        k= n-j
        if i >= (2**k):
            S = S + '1'
            i = i - 2**k
        else:
            S = S + '0'
    return(S)

# Returns all the binary strings of dimension n in order.

def estados(n):
    N = 2**n
    L = []
    for i in range(0,N):
        L.append(bins(i,n))
    return(L)

## The balancing set problem

One of the main problems we must deal with when analysing the GPK algorithm is the balancing problem. Let $f:\mathbb{F}_2^n\to\mathbb{F}_2$ be a Boolean function and $\widehat{f}$ be its Fourier--Hadamard transform, then we should remember that the balancing set of $S = supp(f)$, $B(S)$, is the set of Boolean vectors $\mathbf{u}\in\mathbb{F}_2^n$ such that $\widehat{f}(\mathbf{u}) = 0$. The constant set of $S$ is $\widehat{f}^{-1}(\{\pm|S|\})$ and $b(S) = \frac{|B(S)|}{|C(S)|}$.

Functions will be given as inputs as lists of the images of the function following the lexicographical orgering.

In [17]:
# Returns the balancing set of S.

def B(S):
    n = len(S[0])
    if S == []:
        B = estados(n)
    else:
        B = []
        for i in estados(n):
            k0=0
            k1=0
            for j in S:
                if prods(i,j) == 0:
                    k0 = k0 + 1
                else:
                    k1 = k1 + 1
            if k0 == k1:
                B.append(i)
    return(B)

# Returns the constant set of S.

def C(S):
    C = []
    n = len(S[0])
    for i in estados(n):
        k0=0
        k1=0
        for j in S:
            if prods(i,j) == 0:
                k0 = k0 + 1
            else:
                k1 = k1 + 1
        if (k0 == len(S)) or (k1 == len(S)):
            C.append(i)
    return(C)

# Returns the balancing number of S.

def b(S):
    c = len(C(S))
    b = int(len(B(S)) /c)
    return(b)

# Computes all subsets of n-strings in lexicographical order.

def subs(n):
    L = []
    N = 2**n
    k = 2**N
    for i in estados(N):
        S = []
        r = N-1
        for l in i:
            if l == '1':
                S.append(estados(n)[r])
                r = r-1
            else:
                r = r-1
        L.append(S)
    return(L)

# Calculates all the sets with B as their balancing set.

def antiB(B):
    n = len(B[0])
    A = []
    for L in subs(n):
        if B(L) == B:
            A.append(L)
    return(A)

# Calculates the Fourier--Hadamard of y for the Boolean function whose support is S.

def fhy(S,y):
    L = 0
    for x in S:
        L = L + (-1)**prods(x,y)
    return(L)

# Calculates the Fourier--Hadamard of the Boolean function whose support is S in vector form.

def fh(S):
    n = len(S[0])
    L = []
    for y in estados(n):
        L.append(fhy(S,y))
    return(L)

# Function that adds one to the j in the list of pairs [i,j] if i = k. Just a utility function for the next function.

def change(k,L):
    for [i,j] in L:
        if i == k:
            L.remove([i,j])
            j = j+1
            L.insert(0,[i,j])
    return(L)

# Function that gives us the Fourier--Hadamard support of the Boolean function whose support is S with multiplicities.

def multifh(S):
    L = []
    l = len(S)
    n = len(S[0])
    for i in range(0,l+1):
        if (i % 2) == (l % 2):
            if i == 0:
                L.append([0,0])
            else:
                L.append([i,0])
                L.append([-i,0])
    for y in estados(n):
        k = 0
        for x in S:
            k = k + (-1)**prods(y,x)
        change(k,L)
    return(L)

# A function that returns all possible balancing set sizes for sets of dimension n.

def nums(n):
    lista = []
    for sub in subs(n)[1:]:
        l = len(B(sub))
        if l not in lista:
            lista.append(b)
    return lista

# A function that returns all possible Fourier--Hadamard support sizes for sets of dimension n.

def antinums(n):
    lista = []
    for i in nums(n):
        lista.append(2**n-i)
    return lista

# Classical analysis of the GPK

We now move on to functions that directly compute the amplitudes of the GPK. It should be noted that the amplitude associated with a marker $\mathbf{y}$ and a basis state $\mathbf{x}$ is:
$$
\alpha_{\mathbf{y}}(\mathbf{z}) = \frac{1}{2^n}\sum_{\mathbf{x}\in\mathbb{F}_2^n} (-1)^{f(\mathbf{x})\cdot \mathbf{y} \oplus \mathbf{x}\cdot \mathbf{z}}. 
$$
However, the amplitudes are computed unnormalised in our algorithms. The rows of the GPK matrix correpond to the markers in lexicographical order and the the columns correspond to the basis states. The final function computes all possible final state amplitudes for the one-dimensional GPK (i.e., the Deutsch--Jozsa algorithm).

In [18]:
# WARNING: all the amplitudes are nonnormalised.

# Function that gives us the amplitudes as a vector after applying GPK(y) to the vectorial function determined by S.

def amps(S,y):
    L = []
    N = len(S)
    n = int(math.log(N,2))
    for z in estados(n):
        A = 0
        k = 0
        for x in estados(n):
            f = S[k]
            A = A + (-1)**(prods(y,f)+prods(x,z))
            k = k+1
        L.append(A)
    return(L)

# Function that returns the matrix of amplitudes (GPK matrix).

def totAmps(S):
    L = []
    m = len(S[0])
    for y in estados(m):
        R = amps(S,y)
        L.append(R)
    return(L)

# Converts a string to a list. Just a utility function.

def str2set(z):
    S = []
    for i in z:
        S.append(i)
    return(S)

# Function that returs the amplitude vectors of all possible functions of dimension n after GPK(1).

def ampys(n):
    L = []
    for z in estados(n):
        S = str2set(z)
        L.append(amps(S,'1'))
    return(L)

## A qiskit implementation of the GPK algorithm

We will present here an implementation of the GPK algorithm in qiskit. The oracle function is used to implement the Boolean function $f:\mathbb{F}_2^n\to\mathbb{F}_2^m$ as a matrix using the reversible gate associated with $f$.
We will also draw the circuit. We use 'reverse_bits', so the bits are drawn in the usual manner as opposed to the predetermined one in qiskit.

In [ ]:
import numpy as np
from qiskit import *
from qiskit.extensions import UnitaryGate
from qiskit.quantum_info.operators import Operator

# We begin by fixing the dimentions of the Boolean function:
n = 3
m = 3

# Define a function.

f = ['000','000','010','010','100','100','110','110']

# We create the oracle matrix.

Uf = np.zeros((2**(n+m),2**(n+m)))
for z in estados(n+m):
    i = int(z,2)
    x = z[0:n] + sums(f[int(z[0:n],2)],z[n:n+m])
    Uf[i][int(x,2)] = 1

# And define the quantum gate from the matrix.

oracle_op = Operator(Uf)

# We choose a marker.
y = '001'

# We create the ciruit.
gpk = QuantumCircuit(n+m,n)

# We reconstruct the marker in the second register.
for i in range(len(y)):
    if y[m-1-i] == '1':
        gpk.x(i)

# We apply Hadamard gates to all the qubits.
for i in range(n+m):
    gpk.h(i)

# Now comes the oracle, so we will set up a separation.
gpk.barrier()

# Oracle.
gpk.unitary(oracle_op, range(n+m),label='Oracle')

# Another separation.
gpk.barrier()

# Hadamard gates.
for i in range(n):
    gpk.h(m+i)

# Measurement.
gpkm = gpk
gpkm.measure(range(m,n+m),range(n))

# Muestra
gpk.draw('mpl',reverse_bits=True)

Finally, we use proceed to implement a simlation and analysis of the previously coded algorithm.

In [ ]:
from qiskit import Aer

backend = Aer.get_backend('statevector_simulator')
job = backend.run(gpk)
result = job.result()
outputstate = result.get_statevector(gpk, decimals=None)
print(outputstate)

We can of course implement an actual simulation

In [ ]:
backend_sim = Aer.get_backend('qasm_simulator')
job_sim = backend_sim.run(transpile(gpk, backend_sim), shots=1024)
result_sim = job_sim.result()
counts = result_sim.get_counts(gpk)
print(counts)

We can also apply the usual visualisation tools

In [14]:
from qiskit.visualization import plot_state_city

plot_state_city(outputstate)

ModuleNotFoundError: No module named 'qiskit'